In [ ]:
import pandas as pd
import numpy as np
import altair as alt
from altair import datum
from altair import Scale
from vega_datasets import data

states = alt.topo_feature(data.us_10m.url, 'states')

In [ ]:
df = pd.read_csv("stage3.csv")

In [ ]:
df.dtypes

In [ ]:
df = df.dropna(subset=['incident_characteristics'])

In [ ]:
df_accidents = df[df['incident_characteristics'].str.contains("accident", case=False)]

In [ ]:
df_accidents['year'] = pd.DatetimeIndex(df_accidents['date']).year
df_accidents['month'] = pd.DatetimeIndex(df_accidents['date']).month

In [ ]:
alt.data_transformers.disable_max_rows()
base1 = alt.Chart(df_accidents).mark_line().encode(
    alt.X('month'),
    y='count(incident_id):Q',
).transform_filter(
   (datum.state == 'California') 
)
alt.layer(base1)

In [ ]:
alt.data_transformers.disable_max_rows()
base1 = alt.Chart(df_accidents).mark_line().encode(
    alt.X('month'),
    y='count(incident_id):Q',
).transform_filter(
   (datum.state == 'Illinois') 
)
alt.layer(base1)

In [ ]:
df_state = pd.read_csv("states.csv")

In [ ]:
df_total = pd.merge(df_accidents, df_state, on = "state")

In [ ]:
 df_gun_state = df_total.loc[df_total['year'] == 2017].groupby( ["state", 'id'])["incident_id"].count().reset_index()

In [ ]:
variable_list = ['incident_id']


alt.Chart(states).mark_geoshape().encode(
    alt.Color(alt.repeat('row'), type='quantitative')
).transform_lookup(
    lookup='id',
    from_=alt.LookupData(df_gun_state, 'id', variable_list)
).properties(
    width=500,
    height=300
).project(
    type='albersUsa'
).repeat(
    row=variable_list
).resolve_scale(
    color='independent'
)

In [ ]:
bar_chart = alt.Chart(df_gun_state).mark_bar().encode(
   y=alt.Y('incident_id:Q'),
   x=alt.X('state:N', sort='-y'))
bar_chart

In [ ]:
df_law = pd.read_csv("law_clean.csv")

In [ ]:
df_law

In [ ]:
df_law_state = pd.merge(df_law, df_state, on = "state")

In [ ]:
 df_law_state_group = df_law_state.groupby( ["state", 'id'])["Law ID"].count().reset_index()

In [ ]:
variable_list = ['Law ID']


alt.Chart(states).mark_geoshape().encode(
    alt.Color(alt.repeat('row'), type='quantitative')
).transform_lookup(
    lookup='id',
    from_=alt.LookupData(df_law_state_group, 'id', variable_list)
).properties(
    width=500,
    height=300
).project(
    type='albersUsa'
).repeat(
    row=variable_list
).resolve_scale(
    color='independent'
)

In [ ]:

stacked_chart = alt.Chart(df_law_state).mark_bar().encode(
    x='state',
    y='count(Law ID)',
    color='Law Class'
)
stacked_chart